TF Estimator is a high-level API that makes it simple to create and train models by encapsulating the functionalities for training, evaluating, predicting and exporting. TensorFlow recently re-branded and released the TF Learn package within TensorFlow under the new name TF Estimator.


TF Estimator interface design is inspired from SciKit Learn, allowing to create the estimator object from different kinds of available models, and then providing four main functions on any kind of estimator: 

estimator.fit()    
estimator.evaluate()   
estimator.predict()     
estimator.export()  



TensorFlow provides following pre-built estimators: 

tf.contrib.learn.KMeansClustering    
tf.contrib.learn.DNNClassifier    
tf.contrib.learn.DNNRegressor    
tf.contrib.learn.DNNLinearCombinedRegressor    
tf.contrib.learn.DNNLinearCombinedClassifier    
tf.contrib.learn.LinearClassifier    
tf.contrib.learn.LinearRegressor 
tf.contrib.learn.LogisticRegressor  




Keras library discussed in the next chapter, provides a convenience function to convert Keras models to Estimators: keras.estimator.model_to_estimator().



The simple workflow in TF Estimator API is as follows: 

Find the pre-built Estimator that is relevant to the problem you are trying to solve. 

Write the function to import the dataset. 

Define the columns in data that contain features. 

Create the instance of the pre-built estimator that you selected in step 1. 

Train the estimator. 

Use the trained estimator to do evaluation or prediction.


# TF Estimator MNIST Example 

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=False
                                  )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels


In [4]:
n_classes = 10
batch_size = 100
n_steps = 1000
learning_rate = 0.01


In [5]:
tf.reset_default_graph()


In [6]:
def model_fn(features, labels, mode):
    """ define the model function
    """
    espec_op = tf.estimator.EstimatorSpec
    
    # features is a dict as per Estimator specifications
    x = features['images']
    
    # define the network
    layer_1 = tf.layers.dense(x, 32)
    layer_2 = tf.layers.dense(layer_1, 32)
    logits = tf.layers.dense(layer_2, n_classes)

    # define predicted classes
    predicted_classes = tf.argmax(logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        espec = espec_op(mode,
                         predictions=predicted_classes
                         )
    else:
        # define loss and optimizer
        entropy_op = tf.nn.sparse_softmax_cross_entropy_with_logits
        loss_op = tf.reduce_mean(entropy_op(logits=logits,
                                            labels=tf.cast(labels,
                                                           dtype=tf.int32)
                                            )
                                 )
        
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        
        train_op = optimizer.minimize(
            loss_op, global_step=tf.train.get_global_step())

        # define accuracy
        accuracy_op = tf.metrics.accuracy(
            labels=labels, predictions=predicted_classes)

        espec = espec_op(mode=mode,
                         predictions=predicted_classes,
                         loss=loss_op,
                         train_op=train_op,
                         eval_metric_ops={'accuracy': accuracy_op}
                         )

    return espec




In [7]:
# create estimator object
model = tf.estimator.Estimator(model_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpsirbeqam', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1c3e445cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)
model.train(train_input_fn, steps=n_steps)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpsirbeqam/model.ckpt.
INFO:tensorflow:loss = 2.4960914, step = 0
INFO:tensorflow:global_step/sec: 476.358
INFO:tensorflow:loss = 1.1709845, step = 100 (0.213 sec)
INFO:tensorflow:global_step/sec: 507.165
INFO:tensorflow:loss = 0.81840396, step = 200 (0.196 sec)
INFO:tensorflow:global_step/sec: 516.254
INFO:tensorflow:loss = 0.72289485, step = 300 (0.194 sec)
INFO:tensorflow:global_step/sec: 530.225
INFO:tensorflow:loss = 0.5350197, step = 400 (0.189 sec)
IN

In [9]:
# evaluate the model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test},
    y=y_test,
    batch_size=batch_size,
    shuffle=False)
model.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-09-21:12:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpsirbeqam/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-09-21:12:21
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8897, global_step = 1000, loss = 0.3926905
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpsirbeqam/model.ckpt-1000


{'accuracy': 0.8897, 'loss': 0.3926905, 'global_step': 1000}